<div style="float:center;margin-left:100px;max-width:100%;">
<img src="https://i.imgur.com/XDC6ZdH.gif"></div>

<strong style="color:#B3005E;font-size:25px;font-family:Georgia;text-align:center;">Why the Recommendation system? </strong>

* Benefits users in finding items of their interest.

* Help item providers in delivering their items to the right user.

* Identity products that are most relevant to users.

* Personalized content.

* Help websites to improve user engagement.

<strong style="color:#B3005E;font-size:25px;font-family:Georgia;text-align:center;">1-User Based Collaborative Filtering
 </strong>


* Collaborative filtering is making recommend according to combination of your experience and experiences of other people.
* First we need to make user vs item matrix.
* Each row is users and each columns are items like movie, product or websites
* Secondly, computes similarity scores between users.
* Each row is users and each row is vector.
* Compute similarity of these rows (users).
* Thirdly, find users who are similar to you based on past behaviours
* Finally, it suggests that you are not experienced before.
* User based collaborative filtering has some problems
* In this system, each row of matrix is user. Therefore, comparing and finding similarity between of them is computationaly hard and spend too much computational power.
* Also, habits of people can be changed. Therefore making correct and useful recommendation can be hard in time.
* In order to solve these problems, lets look at another recommender system that is item based collaborative filtering

<strong style="color:#B3005E;font-size:25px;font-family:Georgia;text-align:center;">2-Item Based Collaborative Filtering
 </strong>

* In this system, instead of finding relationship between users, used items like movies or stuffs are compared with each others.
* In user based recommendation systems, habits of users can be changed. This situation makes hard to recommendation. However, in item based recommendation systems, movies or stuffs does not change. Therefore recommendation is easier.
* On the other hand, there are almost 7 billion people all over the world. Comparing people increases the computational power. However, if items are compared, computational power is less.
* In item based recommendation systems, we need to make user vs item matrix that we use also in user based recommender systems.
* Each row is user and each column is items like movie, product or websites.
* However, at this time instead of calculating similarity between rows, we need to calculate similarity between columns that are items like movies or stuffs.


<div style="float:center;margin-left:100px;max-width:100%;">
<img src="https://i.imgur.com/11r6q9d.jpg"></div>

<div class="inner_cell">
<div class="text_cell_render border-box-sizing rendered_html">
<p></p><div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" data-toggle="list" role="tab" aria-controls="home" style = "background-color:#FCFFE7;font-family:Georgia;color:#B3005E;font-size:200%;text-align:LEFT;border-radius:20px 40px;overflow:hidden;border-style:dotted;border-width:1.8px;border-color:#B3005E;">...Table of Contents...</h3>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#1" role="tab" aria-controls="home" target="_self" style = "color:#000000; font-family:Charter;font-size:120%;">1-User Based Collaborative Filtering <span class="badge badge-primary badge-pill">1</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#2" role="tab" aria-controls="messages" target="_self" style = "color:#000000; font-family:Charter;font-size:120%;">Practical to Bring Watched Movies<span class="badge badge-primary badge-pill">2</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#3" role="tab" aria-controls="settings" target="_self" style = "color:#000000; font-family:Charter;font-size:120%;">2- Item Based Collaborative Filtering<span class="badge badge-primary badge-pill">3</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#4" role="tab" aria-controls="settings" target="_self" style = "color:#000000; font-family:Charter;font-size:120%;">Making Movie Recommendations<span class="badge badge-primary badge-pill">4</span></a> 
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#5" role="tab" aria-controls="settings" target="_self" style = "color:#000000; font-family:Charter;font-size:120%;"> ^^^^^<span class="badge badge-primary badge-pill">5</span></a>
</div>
</div>
</div>


In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

<a id = "1"></a>
<strong style="color:#B3005E;font-size:30px;font-family:Georgia;text-align:center;">1-User Based Collaborative Filtering
 </strong>
 

In [2]:

movie = pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')
rating = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv')
    

In [3]:
df = movie.merge(rating, how="left", on="movieId")

def check_df(dataframe, head=5):
    print(" SHAPE ".center(70,'-'))
    print('Rows: {}'.format(dataframe.shape[0]))
    print('Columns: {}'.format(dataframe.shape[1]))
    print(" TYPES ".center(70,'-'))
    print(dataframe.dtypes)
    print(" HEAD ".center(70,'-'))
    print(dataframe.head(head))
    print(" TAIL ".center(70,'-'))
    print(dataframe.tail(head))
    print(" MISSING VALUES ".center(70,'-'))
    print(dataframe.isnull().sum())
    print(" DUPLICATED VALUES ".center(70,'-'))
    print(dataframe.duplicated().sum())
    print(" DESCRIBE ".center(70,'-'))
    print(dataframe.describe().T)
    
check_df(df)

------------------------------- SHAPE --------------------------------
Rows: 20000797
Columns: 6
------------------------------- TYPES --------------------------------
movieId        int64
title         object
genres        object
userId       float64
rating       float64
timestamp     object
dtype: object
-------------------------------- HEAD --------------------------------
   movieId             title                                       genres  userId  rating            timestamp
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy     3.0     4.0  1999-12-11 13:36:47
1        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy     6.0     5.0  1997-03-13 17:50:52
2        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy     8.0     4.0  1996-06-05 13:37:51
3        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy    10.0     4.0  1999-11-25 02:44:47
4        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|

In [4]:
def create_user_movie_df():
    comment_counts = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 1000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df
user_movie_df = create_user_movie_df()

<a id = "2"></a>
<strong style="color:#B3005E;font-size:25px;font-family:Georgia;text-align:center;">Practical to Bring Watched Movies </strong>

In [5]:
# Oneri yapilacak kullanici
random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=45).values)
random_user
# user hangi filmelri izlemis, ne puan vermis bul 
# secili user ile benzer begenme davranisi sergileyenleri bul
# skor hesabi yap
# Ve bu dogrultuda tavsiyede bulun

28941

In [6]:
# veri setinden kullanicimizi ilgilendiren bolumu seceleim
# kullanicinin izledigi filmler.. 
random_user_df = user_movie_df[user_movie_df.index == random_user]

# NA olmayanlari getir    
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
movies_watched #Izlenen filmler listesi

['Ace Ventura: Pet Detective (1994)',
 'Ace Ventura: When Nature Calls (1995)',
 'Aladdin (1992)',
 'American President, The (1995)',
 'Apollo 13 (1995)',
 'Babe (1995)',
 'Bullets Over Broadway (1994)',
 'Clueless (1995)',
 'Disclosure (1994)',
 'Forrest Gump (1994)',
 'Four Weddings and a Funeral (1994)',
 'Home Alone (1990)',
 'Jurassic Park (1993)',
 'Like Water for Chocolate (Como agua para chocolate) (1992)',
 'Little Women (1994)',
 "Mr. Holland's Opus (1995)",
 'Mrs. Doubtfire (1993)',
 'Much Ado About Nothing (1993)',
 "Muriel's Wedding (1994)",
 'Nine Months (1995)',
 'Operation Dumbo Drop (1995)',
 'Piano, The (1993)',
 'Postman, The (Postino, Il) (1994)',
 'Ready to Wear (Pret-A-Porter) (1994)',
 'Remains of the Day, The (1993)',
 'Sabrina (1995)',
 "Schindler's List (1993)",
 'Secret Garden, The (1993)',
 'Sense and Sensibility (1995)',
 'Shadowlands (1993)',
 'Silence of the Lambs, The (1991)',
 'Star Trek: Generations (1994)',
 'Stargate (1994)']

In [7]:
#kullanicimiz 33 film izlemis
len(movies_watched)

33

In [8]:
# kullanicimizin "Silence of the Lambs, The (1991)" filmine kac puan verdigini merak ediyoruz.
user_movie_df.loc[user_movie_df.index == random_user,user_movie_df.columns == "Silence of the Lambs, The (1991)"]

title,"Silence of the Lambs, The (1991)"
userId,
28941.0,1.0


 <a id = "3"></a>

 <strong style="color:#B3005E;font-size:20px;font-family:Georgia;text-align:center;">Other Users Watching the Same Movies  </strong>
 

In [9]:
# Ayni filleri izlemis kullanicilarin bilgilerine eriselim
movies_watched_df = user_movie_df[movies_watched]
movies_watched_df.iloc[0:5, 0:5]

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Aladdin (1992),"American President, The (1995)",Apollo 13 (1995)
userId,,,,,
1.0,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN
4.0,NaN,3.0,NaN,NaN,NaN
5.0,NaN,NaN,5.0,5.0,5.0


In [10]:
# Buradan sadece kullanicimizla en az 20 ortak filmi izlemis olanlari secmek istiyoruz.
# Bunun icin digel kullanicilarimizin kacar film izledigi bilgisine erismek istiyoruz.
# Burada belirlenecek olan esik degerler gorecelidir. Degisiklik gosterebilir.
user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId", "movie_count"]
#  En az 20 ortak film
user_movie_count[user_movie_count["movie_count"] > 20].sort_values("movie_count", ascending=False)


,userId,movie_count
94230,94231.0,33
100398,100399.0,33
118204,118205.0,33
15918,15919.0,33
124051,124052.0,33
...,...,...
79214,79215.0,21
79174,79175.0,21
9105,9106.0,21
78515,78516.0,21


In [11]:
# Mesela kullanicimiizin izledigi butun filmleri izlemis olan diger kullanicilari gormek istersek.
user_movie_count[user_movie_count["movie_count"] == 33].count()

userId         17
movie_count    17
dtype: int64

In [12]:
users_same_movies = user_movie_count[user_movie_count["movie_count"] > 20]["userId"]

 <a id = "4"></a>

 <strong style="color:#B3005E;font-size:25px;font-family:Georgia;text-align:center;">Determination of Similarity  </strong>

In [13]:
# Kullanicimiz ve digerlerinin verilerini korelasyonu gormek amaciyla birlestiricez
# En benzerleri tespit edicez 
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                      random_user_df[movies_watched]])

In [14]:
# Daha kolay islem yapabilmek icin Sutunlara kullanicilari alliyoruz
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
# Butun veriyi birlestirdikten sonra korelasyonunu alip daha okunakli bir hale getirelim
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
corr_df.head() 
## Tum kullanicilar acisindan korelasyonlar.

,user_id_1,user_id_2,corr
0,28866.0,67756.0,-0.936065
1,60562.0,37121.0,-0.915003
2,34103.0,80593.0,-0.898718
3,62575.0,21398.0,-0.896612
4,117826.0,48416.0,-0.883969


In [15]:
# Bizim ihtiyacimiz sadece secili kullanicimizi ilgilendiren korelasyonlar.
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.70)][["user_id_2", "corr"]].reset_index(drop=True)

# Kullanicimiz ile %70 ve uzeri korelasyona sahip diger kullanicilari gorelim. Uygulanan filtre gorece degisir.
top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)

top_users.head(20)
# Kullanicimiz ile en benzer begenme davranisi gosteren kullanicilarimizi tespit ettik,
# Simdi bu kullanicilarin hangi filme kac puan verdiklerini tespit edelim

,userId,corr
12,28941.0,1.000000
11,45158.0,0.800749
10,101628.0,0.790405
9,127259.0,0.763925
8,9783.0,0.747942
7,93089.0,0.738536
6,136259.0,0.737177
5,136259.0,0.737177
4,67346.0,0.732943
3,67346.0,0.732943


In [16]:
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
# Kullanicimiz ile en yuksek korelasyona sahip kullanicilar ve filmlere verdikleri puanlar
top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]
top_users_ratings.head(10) 

,userId,corr,movieId,rating
33,45158.0,0.800749,1,1.5
34,45158.0,0.800749,3,1.0
35,45158.0,0.800749,17,3.5
36,45158.0,0.800749,19,2.0
37,45158.0,0.800749,22,3.0
38,45158.0,0.800749,25,4.0
39,45158.0,0.800749,31,3.0
40,45158.0,0.800749,32,2.5
41,45158.0,0.800749,36,4.0
42,45158.0,0.800749,39,3.5


<a id = "5"></a>

<strong style="color:#B3005E;font-size:25px;font-family:Georgia;text-align:center;">Weighted Average Recommendation Score Calculation </strong>


In [17]:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']

recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"}).reset_index()
recommendation_df.head(10)

,movieId,weighted_rating
0,1,2.203838
1,2,1.224009
2,3,0.934528
3,4,1.633519
4,5,1.832357
5,6,1.436111
6,7,2.724309
7,10,2.124006
8,11,2.428406
9,13,0.738536


In [18]:
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.5].sort_values("weighted_rating", ascending=False)
movies_to_be_recommend.head(10)

,movieId,weighted_rating
26,30,3.952023
237,326,3.952023
365,501,3.952023
44,53,3.952023
186,263,3.952023
168,242,3.952023
575,933,3.777725
844,1348,3.692678
3452,25850,3.664714
2687,5799,3.664714


In [19]:
movie = pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')
movies_to_be_recommend.merge(movie[["movieId", "title"]])

,movieId,weighted_rating,title
0,30,3.952023,Shanghai Triad (Yao a yao yao dao waipo qiao) ...
1,326,3.952023,To Live (Huozhe) (1994)
2,501,3.952023,Naked (1993)
3,53,3.952023,Lamerica (1994)
4,263,3.952023,Ladybird Ladybird (1994)
...,...,...,...
59,1230,3.543752,Annie Hall (1977)
60,908,3.543752,North by Northwest (1959)
61,4174,3.535917,Avalon (1990)
62,903,3.510809,Vertigo (1958)


<a id = "3"></a>

<strong style="color:#B3005E;font-size:30px;font-family:Georgia;text-align:center;">2- Item Based Collaborative Filtering
 </strong>
 
 <strong style="color:#B3005E;font-size:20px;font-family:Georgia;text-align:center;">Creation of User Movie Df 
 </strong>

In [20]:
df["title"].nunique()

27262

In [21]:
df["title"].value_counts().head()

Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
Name: title, dtype: int64

In [22]:
comment_counts = pd.DataFrame(df["title"].value_counts())
rare_movies = comment_counts[comment_counts["title"] <= 1000].index
common_movies = df[~df["title"].isin(rare_movies)]
common_movies.shape
common_movies["title"].nunique()

3159

In [23]:
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")

user_movie_df.shape

(138493, 3159)

<a id = "7"></a>

<strong style="color:#B3005E;font-size:25px;font-family:Georgia;text-align:center;">Making Movie Recommendations
 </strong>


In [24]:
# Matrix filmiini izleyen kullanicilara asagidaki oneriler yapilabilir.
movie_name = "Matrix, The (1999)"
movie_name = user_movie_df[movie_name]
user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

title
Matrix, The (1999)                                           1.000000
Matrix Reloaded, The (2003)                                  0.516906
Matrix Revolutions, The (2003)                               0.449588
Animatrix, The (2003)                                        0.367151
Blade (1998)                                                 0.334493
Terminator 2: Judgment Day (1991)                            0.333882
Minority Report (2002)                                       0.332434
Edge of Tomorrow (2014)                                      0.326762
Mission: Impossible (1996)                                   0.320815
Lord of the Rings: The Fellowship of the Ring, The (2001)    0.318726
dtype: float64

In [25]:
movie_name = "Ocean's Twelve (2004)"
movie_name = user_movie_df[movie_name]
user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

title
Ocean's Twelve (2004)                                 1.000000
Ocean's Thirteen (2007)                               0.681654
Ocean's Eleven (2001)                                 0.551280
Eddie (1996)                                          0.474808
National Treasure: Book of Secrets (2007)             0.474230
Eagle Eye (2008)                                      0.473061
Pirates of the Caribbean: On Stranger Tides (2011)    0.472446
Ocean's Eleven (a.k.a. Ocean's 11) (1960)             0.470412
Analyze That (2002)                                   0.459010
Bad Boys II (2003)                                    0.458827
dtype: float64

In [26]:
# Hadi rastgele bir film ismi secip secilen filme onerilebilecek filmleri tespit edelim.
movie_name = pd.Series(user_movie_df.columns).sample(1).values[0]
movie_name = user_movie_df[movie_name]
user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

title
Iron Man 2 (2010)                                   1.000000
Iron Man (2008)                                     0.642771
Iron Man 3 (2013)                                   0.635673
Mrs. Winterbourne (1996)                            0.565232
Fantastic Four: Rise of the Silver Surfer (2007)    0.565207
Farinelli: il castrato (1994)                       0.546955
Avengers, The (2012)                                0.546732
Thor (2011)                                         0.544901
Transformers: Revenge of the Fallen (2009)          0.543408
Ghost Rider (2007)                                  0.537334
dtype: float64

In [27]:
# Asagidaki fonksiyon ile girdigimiz keyword e gore film secimi yapabiliriz.
def check_film(keyword, user_movie_df):
    return [col for col in user_movie_df.columns if keyword in col]

check_film("Insomnia", user_movie_df)

['Insomnia (1997)', 'Insomnia (2002)']

In [28]:
check_film('Lord', user_movie_df)

['Lord of Illusions (1995)',
 'Lord of War (2005)',
 'Lord of the Flies (1963)',
 'Lord of the Flies (1990)',
 'Lord of the Rings, The (1978)',
 'Lord of the Rings: The Fellowship of the Ring, The (2001)',
 'Lord of the Rings: The Return of the King, The (2003)',
 'Lord of the Rings: The Two Towers, The (2002)']

<p style="padding:15px;
background-color:#f4ebdc;
margin:0; color:#B3005E; border:2px dotted #C689C6; font-family:Charter; font-weight: bold; font-size:250%; text-align:center; overflow:hidden; font-weight:500">If you like this then please UPVOTE this 😄 Your opinions and suggestions are very important to me...<img src="https://media.giphy.com/media/WUlplcMpOCEmTGBtBW/giphy.gif" width="100"> <div style="float:center;margin-left:270px;max-width:50%;">

<p style="padding:15px;
background-color:#f4ebdc;
margin:0; color:#B3005E; border:2px dotted #C689C6; font-family:Charter; font-weight: bold; font-size:250%; text-align:center; overflow:hidden; font-weight:500">
FOR MORE:</p> 

https://www.linkedin.com/in/serdar-ozturk/

https://github.com/StanleyHopson

https://medium.com/@serdar.f95

<p style="padding:15px;
background-color:#f4ebdc;
margin:0; color:#B3005E; border:2px dotted #C689C6; font-family:Charter; font-weight: bold; font-size:250%; text-align:center; overflow:hidden; font-weight:500">CREDITS:</p> 

[https://numpy.org/](http://)

[https://seaborn.pydata.org/](http://)

[https://pandas.pydata.org](http://)

[https://pandas.pydata.org/](http://)

[https://learning.miuul.com](http://)